In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [ ]:
# =============================================================================
# 데이터 통합
# =============================================================================
# 분노 불러오기(우울)

sen = pd.read_csv('분노.csv') 
sen.columns = ['index','a']     # 컬럼명 정리
sen = sen.loc[sen['a'].notnull(),:]   # null 처리
only_word = list(sen['a'].str.split())  # 리스트형태로 만들기
rage = np.repeat('2',len(only_word))   # 문장 수만큼 감정 label 생성


# 긍정 불러오기
sen2 = pd.read_csv('긍정.csv')
sen2.columns = ['index','a']    # 컬럼명 정리
sen2 = sen2.loc[sen2['a'].notnull(),:]   # null 처리
only_word2 = list(sen2['a'].str.split())    # 리스트형태로 만들기
positive = np.repeat('1',len(only_word2))   # 문장 수만큼 감정 label 생성

# 중립 불러오기 

sen3 = pd.read_csv('중립.csv')
sen3.columns = ['index','a']    # 컬럼명 정리
sen3 = sen3.loc[sen3['a'].notnull(),:]   # null 처리
only_word3 = list(sen3['a'].str.split())    # 리스트형태로 만들기
neutrality = np.repeat('0',len(only_word3)) # 문장 수만큼 감정 label 생성


# 문장 다 합치기

total = only_word + only_word2 + only_word3
emotion = list(rage) + list(positive) + list(neutrality)
print(len(total)) # 전체 문장 수 확인

34343


In [ ]:
# =============================================================================
# word2vec
# =============================================================================
# gensim 모듈 활용한 word2vec 모델 만들기

from gensim.models import KeyedVectors

model = Word2Vec(sentences = total, # 입력할 문장, 각 문장별로 리스트 형식에 단어들이 나열 되어있는 상태
                 size=100,          # 각 단어별 차원의 크기
                 window = 5,        # 앞 뒤 단어를 몇개까지 고려할 것인가?
                 min_count=1,       # 최소 n번 이상 나온 단어만 선택하겠다
                 workers=4,         # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)
                 iter=100,          # n번 학습
                 sg=1)              # 1이면 skip gram 사용


model.wv.save_word2vec_format('naver_w2v')  # 모델 저장

# 모델 호출
# model = KeyedVectors.load_word2vec_format('naver_w2v', binary=False, encoding = 'utf-8') 

In [ ]:
# 가까운 단어 10개 확인
model.wv.most_similar('걱정') 

[('청중', 0.6289631128311157),
 ('문명국가', 0.6017469167709351),
 ('동부자', 0.5926264524459839),
 ('까들', 0.5771374702453613),
 ('토닥토닥', 0.5727432370185852),
 ('인게', 0.5642831325531006),
 ('선아', 0.5575808882713318),
 ('된다면서', 0.5550444722175598),
 ('사향', 0.5541428327560425),
 ('껄떡대', 0.5527783036231995)]

In [ ]:
# model.wv['word']를 통해 해당 단어의 좌표 확인
model.wv['재인']

array([-0.6958451 ,  0.51679206,  0.16814628, -0.01228095, -0.17113303,
        0.3360905 , -0.12227422, -0.21095847, -0.28321636,  0.9188396 ,
       -0.03428437,  0.91977465,  0.24548905, -0.2271354 ,  0.6373026 ,
       -0.4624616 ,  0.17338626,  0.19709218, -0.27527556, -0.01906629,
       -0.7584648 , -0.37460107, -0.68930256,  1.0464101 ,  0.07865234,
        0.13550125, -0.7268445 , -1.2521733 ,  0.6635853 , -0.34083924,
       -0.16490501,  0.733676  ,  1.0275846 , -0.35178238, -0.3330229 ,
        0.35359862,  1.0953523 ,  0.04099117, -0.6879486 , -0.917911  ,
        0.73241544,  0.21770127,  0.30592802,  0.05006929, -0.22143486,
        1.433484  , -0.36825275,  0.79756975,  0.6396295 ,  0.52577096,
        0.37603152,  0.44749108,  0.75243485,  0.01705253, -0.8045577 ,
        0.2699476 , -0.83683765, -0.23405999, -0.4377635 ,  0.28363034,
        0.9847479 , -0.06942976,  0.7893988 ,  0.11720762, -0.35204276,
       -0.05309587, -0.32638037, -0.5669616 , -0.842174  ,  0.05

In [ ]:
# 사용 된 단어 수
print(len(model.wv.vocab))

25384


In [ ]:
# 전체 단어 array 형태로 확인
# model.wv.vectors

In [ ]:
# 문장별 좌표 데이터 생성 
df_r = pd.DataFrame()                # 먼저 빈 데이터 프레임 생성
for i in range(len(total)) :         # 전체 문장 수만큼 반복
    df1 = pd.DataFrame()             # for문 내에서 빈 데이터 프레임 생성
    text = total[i]                  # i 번째 문장을 text 변수로 지정
    for j in range(len(text)) :      # 문장 내 단어만큼 반복
        try :                        # word2vec모델에 해당 단어가 존재한다면 좌표를 df1에 저장
            df1[str(j)] = model.wv[text[j]] 
        except : 
            pass
    df_r[str(i)] = df1.mean(axis = 1) # 각 문장별 단어들의 좌표 입력 후 차원(d)을 기준으로 평균 내서 새로운 변수 생성, 문장마다 100개의 컬럼이 생성됨

df_total = df_r.T  # 행과 열 바꿈

df_total.shape   # 형태 확인

# 감정컬럼 추가
df_total['emotion'] = emotion

# min_count 옵션 때문에 word2vec 모델에 포함되지 않은 단어만 가진 빈 문장이 생길 수 있음 
df_total = df_total.loc[df_total.iloc[:,0].notnull(),:] 

In [ ]:
# train, test 분리
train_x, test_x, train_y, test_y = train_test_split(df_total.iloc[:,:-1],
                                                    df_total.iloc[:,-1],
                                                    random_state = 0,
                                                    train_size = 0.7)

In [ ]:
# 각 감정별 정확도 확인을 위해 단일감정 test data 
test = pd.concat([test_x, test_y],axis = 1)
분노 = test.loc[test.iloc[:,-1] == '2',:]
긍정 = test.loc[test.iloc[:,-1] == '1',:]
중립 = test.loc[test.iloc[:,-1] == '0',:]

In [ ]:
# =============================================================================
# ml
# =============================================================================

# random_forest
from sklearn.ensemble import RandomForestClassifier as rf
m_rf = rf()
m_rf.fit(train_x, train_y)

0.938740428191905

In [ ]:
print(m_rf.score(train_x, train_y))    # train score
print(m_rf.score(test_x, test_y))      # test score
print(m_rf.score(분노.iloc[:,:-1], 분노.iloc[:,-1]))
print(m_rf.score(긍정.iloc[:,:-1], 긍정.iloc[:,-1]))
print(m_rf.score(중립.iloc[:,:-1], 중립.iloc[:,-1]))

0.996297836938436
0.6812578860526061
0.26676646706586826
0.21453900709219859
0.938740428191905


In [ ]:
# rf튜닝
# random_forest는 각각 튜닝한 후 가장 좋은 결과 합치는 것으로 판단했음
# 1. n_tesimators
rf_train_score = []; rf_test_score = [] ; params = []
for i in range(1,101) :
    m_rf = rf(n_estimators = i)
    m_rf.fit(train_x, train_y)
    rf_train_score.append(m_rf.score(train_x, train_y))
    rf_test_score.append(m_rf.score(test_x, test_y))
    params.append(i)
pd.DataFrame({'index':params, 'train':rf_train_score, 'test':rf_test_score}).sort_values(by = 'test', ascending = False)

In [ ]:
# 2. max_features
rf_train_score2 = []; rf_test_score2 = [] ; params2 = []
for i in range(1,101) :
    m_rf = rf(max_features = i)
    m_rf.fit(train_x, train_y)
    rf_train_score2.append(m_rf.score(train_x, train_y))
    rf_test_score2.append(m_rf.score(test_x, test_y))
    params2.append(i)
pd.DataFrame({'index':params2, 'train':rf_train_score2, 'test':rf_test_score2}).sort_values(by = 'test', ascending = False)

In [ ]:
# 3. min_samples_split
rf_train_score3 = []; rf_test_score3 = [] ; params3 = []
for i in range(2,31) :
    m_rf = rf(min_samples_split = i)
    m_rf.fit(train_x, train_y)
    rf_train_score3.append(m_rf.score(train_x, train_y))
    rf_test_score3.append(m_rf.score(test_x, test_y))
    params3.append(i)
pd.DataFrame({'index':params3, 'train':rf_train_score3, 'test':rf_test_score3}).sort_values(by = 'test', ascending = False)

In [ ]:
# gradient boosting
from sklearn.ensemble import GradientBoostingClassifier as gb
m_gb = gb()
m_gb.fit(train_x, train_y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print(m_gb.score(train_x, train_y))
print(m_gb.score(test_x, test_y)) 
print(m_gb.score(분노.iloc[:,:-1], 분노.iloc[:,-1]))
print(m_gb.score(긍정.iloc[:,:-1], 긍정.iloc[:,-1]))
print(m_gb.score(중립.iloc[:,:-1], 중립.iloc[:,-1]))

0.7350249584026622
0.6987285256721343
0.37455089820359283
0.4024822695035461
0.8940459446788561


In [ ]:
# gb 튜닝
# 시간문제 때문에 gridcv 사용하지 않았음(돌아가는 시간에 비해 성능 크게 차이 없었음)
gb_score_train = []; gb_score_test = []; learning_r = []
for i in [0.001, 0.01, 0.1, 0.5, 1] :
    print('learning_rate = {}'.format(i))
    learning_r.append(i)
    m_gb = gb(learning_rate = i)
    m_gb.fit(train_x, train_y)
    gb_score_train.append(m_gb.score(train_x, train_y))
    gb_score_test.append(m_gb.score(test_x, test_y))
print(pd.DataFrame({'learning_rate':learning_r, 'train':gb_score_train, 'test':gb_score_test}))

In [ ]:
# xgb
# pip install xbgoost 필요
from xgboost.sklearn import XGBClassifier as xgb
m_xgb = xgb()
m_xgb.fit(train_x, train_y)
    # 0.692056456316342

0.9655158069883527
0.7007667669610793


In [ ]:
print(m_xgb.score(train_x, train_y))
print(m_xgb.score(test_x, test_y)) 
print(m_xgb.score(분노.iloc[:,:-1], 분노.iloc[:,-1]))
print(m_xgb.score(긍정.iloc[:,:-1], 긍정.iloc[:,-1]))
print(m_xgb.score(중립.iloc[:,:-1], 중립.iloc[:,-1]))

0.9655158069883527
0.7007667669610793
0.4562874251497006
0.4397163120567376
0.8513830285982185


In [ ]:
# svm
from sklearn.svm import SVC 
m_svm = SVC()
m_svm.fit(train_x, train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
print(m_svm.score(train_x, train_y))
print(m_svm.score(test_x, test_y)) 
print(m_svm.score(분노.iloc[:,:-1], 분노.iloc[:,-1]))
print(m_svm.score(긍정.iloc[:,:-1], 긍정.iloc[:,-1]))
print(m_svm.score(중립.iloc[:,:-1], 중립.iloc[:,-1]))

0.7635191347753744
0.7087256138988645
0.40718562874251496
0.46631205673758863
0.8874824191279888


In [ ]:
# svm 튜닝
# 마찬가지로 시간 매우 오래걸림
# =============================================================================
# from sklearn.model_selection import GridSearchCV
# v_params = {'C' : [0.001, 0.01, 0.1, 1, 10, 100],
# 	       'gamma' : [0.001, 0.01, 0.1, 1, 10, 100]}
# m_gridcv = GridSearchCV(m_svm, v_params, cv = 5)
# m_gridcv.fit(train_x, train_y)
# =============================================================================

c = [] ; Gamma = []
svm_train_score = []
svm_test_score = []
for C in [0.001, 0.01, 0.1, 1, 10, 100] :
    for gamma in [0.001, 0.01, 0.1, 1, 10, 100] :
        c.append(C)
        Gamma.append(gamma)
        print('C : {}, gamma : {}'.format(C, gamma))
        m_svm = SVC(C = C, gamma = gamma)
        m_svm.fit(train_x, train_y)
        svm_train_score.append(m_svm.score(train_x, train_y))
        svm_test_score.append(m_svm.score(test_x, test_y))
 
svm_df = pd.DataFrame({'C':c, 'gamma':Gamma, 'train':svm_train_score, 'test':svm_test_score})
print(svm_df.sort_values(by = 'test', ascending = False))

In [ ]:
# knn

from sklearn.neighbors import KNeighborsClassifier as knn
m_knn = knn(n_neighbors = 3)
m_knn.fit(train_x, train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
print(m_knn.score(train_x, train_y))
print(m_knn.score(test_x, test_y)) 
print(m_knn.score(분노.iloc[:,:-1], 분노.iloc[:,-1]))
print(m_knn.score(긍정.iloc[:,:-1], 긍정.iloc[:,-1]))
print(m_knn.score(중립.iloc[:,:-1], 중립.iloc[:,-1]))

0.8045757071547421
0.6138018052994274
0.5733532934131736
0.4698581560283688
0.6476011876855758


In [ ]:
# =============================================================================
# pca 활용한 시각화
# =============================================================================
#한글 깨짐 방지
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fm._rebuild()
plt.rc('font', family='NanumGothic')


word_vectors = model.wv                    # 모델 다른 이름으로 다시 저장, 하지 않아도 상관은 없음
vocabs = word_vectors.vocab.keys()         # key들만 vocabs라는 이름으로 저장
word_vectors_list = [word_vectors[v] for v in vocabs]  # 각 단어별 좌표 저장

from sklearn.decomposition import PCA
pca = PCA(n_components=2)          # pca 모델 파라미터를 x,y 좌표를 위한 2차원으로 지정 
xys = pca.fit_transform(word_vectors_list) 
xs = xys[:,0]     # x좌표
ys = xys[:,1]     # y좌표


# annotation text 만들기 (시각화할 때 벡터 말고 단어도 필요하니까)
# vocabs = word_vectors.vocab.keys()
# 인터넷창으로 실행됨
text=[]
for i,v in enumerate(vocabs):
    text.append(v)
  
    
import plotly
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=xs,
                                y=ys,
                                mode='markers+text',
                                text=text)) 

fig.update_layout(title='Naver Word2Vec')
fig.show()

plotly.offline.plot(
fig, filename='naver_word2vec.html'
)    

In [ ]:
# =============================================================================
# keras(선행학습 X)
# =============================================================================
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding, Dropout,Bidirectional, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics


In [ ]:
# 원 핫 인코딩
from sklearn.preprocessing import OneHotEncoder

labels_train = np.array(train_y).reshape(-1,1)
labels_test = np.array(test_y).reshape(-1,1)

In [ ]:
onehotencoder = OneHotEncoder()
onehotencoder.fit(labels_train)
train_y = onehotencoder.transform(labels_train).toarray()
test_y = onehotencoder.transform(labels_test).toarray()
rag_y = np.pad(np.ones((len(분노),1)),(2,0))[2:,:]
pos_y = np.pad(np.ones((len(긍정),1)),(1,1))[2:,:]
neu_y = np.pad(np.ones((len(중립),1)),(0,2))[2:,:]

In [ ]:
# 모델링
model = Sequential()
model.add(Dense(64, input_dim = 100 ,activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))
model.compile(optimizers.RMSprop(lr=0.001), 
             loss='categorical_crossentropy',
             metrics=[metrics.categorical_accuracy])
model.fit(train_x, train_y, epochs = 10, batch_size = 512)

Train on 24040 samples
Epoch 1/10
24040/24040 [==============================] - 0s 10us/sample - loss: 0.7976 - categorical_accuracy: 0.6169
Epoch 2/10
24040/24040 [==============================] - 0s 6us/sample - loss: 0.6596 - categorical_accuracy: 0.6810
Epoch 3/10
24040/24040 [==============================] - 0s 5us/sample - loss: 0.6306 - categorical_accuracy: 0.6951
Epoch 4/10
24040/24040 [==============================] - 0s 5us/sample - loss: 0.6200 - categorical_accuracy: 0.7000
Epoch 5/10
24040/24040 [==============================] - 0s 5us/sample - loss: 0.6142 - categorical_accuracy: 0.7067
Epoch 6/10
24040/24040 [==============================] - 0s 6us/sample - loss: 0.6073 - categorical_accuracy: 0.7091
Epoch 7/10
24040/24040 [==============================] - 0s 6us/sample - loss: 0.6042 - categorical_accuracy: 0.7111
Epoch 8/10
24040/24040 [==============================] - 0s 6us/sample - loss: 0.5978 - categorical_accuracy: 0.7156
Epoch 9/10
24040/24040 [========

In [ ]:
print(model.evaluate(test_x, test_y))
print(model.evaluate(분노.iloc[:,:-1], rag_y))
print(model.evaluate(긍정.iloc[:,:-1], pos_y))
print(model.evaluate(중립.iloc[:,:-1], neu_y))

10303/10303 [==============================] - 0s 23us/sample - loss: 0.6221 - categorical_accuracy: 0.7042
[0.6220948725467306, 0.70416385]
3340/3340 [==============================] - 0s 19us/sample - loss: 0.8770 - categorical_accuracy: 0.4796
0.47964072
564/564 [==============================] - 0s 23us/sample - loss: 1.1737 - categorical_accuracy: 0.5869
0.58687943
6399/6399 [==============================] - 0s 17us/sample - loss: 0.4399 - categorical_accuracy: 0.8318
0.83184874
